In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def rms(x):
    return np.sqrt(np.mean(x ** 2))

def pp(x):
    return np.max(x) - np.min(x)

FS = 4096
T_START = 100
WINDOW = FS*2
LENGTH_SECONDS = 100

def load_dataset(name):
    columns = ['v_in', 'rpm', 'x', 'y', 'z']
    d = pd.read_csv(
        name, sep=',',  
        skiprows=T_START*FS, nrows=FS*LENGTH_SECONDS, 
        header=None, names=columns
    )
    d['t'] = d.index * (1 / FS)
    d['f'] = d['rpm'] / 60
    d['mag'] = np.sqrt(d['x'] ** 2 + d['y'] ** 2 + d['z'] ** 2)
    
    # Peak-to-peak
    d['x_pp'] = d['x'].rolling(window=WINDOW).apply(pp, raw=True)
    d['y_pp'] = d['y'].rolling(window=WINDOW).apply(pp, raw=True)
    d['z_pp'] = d['z'].rolling(window=WINDOW).apply(pp, raw=True)
    d['mag_pp'] = d['mag'].rolling(window=WINDOW).apply(pp, raw=True)
    
    # RMS
    d['x_rms'] = d['x'].rolling(window=WINDOW).apply(rms, raw=True)
    d['y_rms'] = d['y'].rolling(window=WINDOW).apply(rms, raw=True)
    d['z_rms'] = d['z'].rolling(window=WINDOW).apply(rms, raw=True)
    d['mag_rms'] = d['z'].rolling(window=WINDOW).apply(rms, raw=True)
    return d[WINDOW:]


shaft = [
    load_dataset('../../datasets/Rotating-shaft/0D.csv'),
    #load_dataset('Rotating-shaft/1D.csv')
    #load_dataset('Rotating-shaft/2D.csv'),
    #load_dataset('Rotating-shaft/3D.csv'),
    load_dataset('../../datasets/Rotating-shaft/4D.csv')
]

In [ ]:
d0 = shaft[0]
d0

In [ ]:
d4 = shaft[1]
d4

In [ ]:
d0[FS*30:FS*30+500].plot(x='t', y=['x', 'y', 'z', 'mag'], subplots=True, sharex=True, figsize=(20, 10))

In [ ]:
d0.plot(x='t', y=['x', 'y', 'z', 'mag'], subplots=True, sharex=True, figsize=(20, 10))

In [ ]:
d4.plot(x='t', y=['x', 'y', 'z', 'mag'], subplots=True, sharex=True, figsize=(20, 10))

In [ ]:
d0.plot(x='t', y=['x_rms','y_rms', 'z_rms', 'mag_rms'], subplots=True, sharex=True, figsize=(20, 12))

In [ ]:
d4.plot(x='t', y=['x_rms','y_rms', 'z_rms', 'mag_rms'], subplots=True, sharex=True, figsize=(20, 12))

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(20, 15))

for i, col in enumerate(('x_rms', 'y_rms', 'z_rms', 'mag_rms')) :
    ax[i].scatter(d0['rpm'], d0[col], s=2)
    ax[i].scatter(d4['rpm'], d4[col], s=2)
    ax[i].set_xlim(660, 860)


#d0.plot(kind='scatter', x='RPM', y='y_rms', figsize=(20, 5), s=2)
#d0.plot(kind='scatter', x='RPM', y='z_rms', figsize=(20, 5), s=2)

In [ ]:
d0.plot(x='t', y='rpm', subplots=True, sharex=True, figsize=(20, 8), xlabel="Time [s]", ylabel="RPM")

In [ ]:
d0.plot(x='t', y='f', subplots=True, sharex=True, figsize=(20, 8), xlabel="Time [s]", ylabel="Frequency [Hz]")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 6))

ax[0].scatter(d0['x_rms'], d0['y_rms'], s=2)
ax[0].scatter(d4['x_rms'], d4['y_rms'], s=2)
ax[0].set_xlabel('X')
ax[0].set_ylabel('Y')
ax[0].grid()

ax[1].scatter(d0['y_rms'], d0['z_rms'], s=2)
ax[1].scatter(d4['y_rms'], d4['z_rms'], s=2)
ax[1].set_xlabel('Y')
ax[1].set_ylabel('Z')
ax[1].grid()

ax[2].scatter(d0['x_rms'], d0['z_rms'], s=2)
ax[2].scatter(d4['x_rms'], d4['z_rms'], s=2)
ax[2].set_xlabel('X')
ax[2].set_ylabel('Z')
ax[2].grid()

plt.show()

In [ ]:
df0 = shaft[0]
fig, ax = plt.subplots(1, 3, figsize=(20, 6))

ax[0].scatter(df0['x'], df0['y'], s=2)
ax[0].set_xlabel('X')
ax[0].set_ylabel('Y')
ax[0].grid()

ax[1].scatter(df0['y'], df0['z'], s=2)
ax[1].set_xlabel('Y')
ax[1].set_ylabel('Z')
ax[1].grid()

ax[2].scatter(df0['x'], df0['z'], s=2)
ax[2].set_xlabel('X')
ax[2].set_ylabel('Z')
ax[2].grid()

plt.show()

In [ ]:
df4 = shaft[4]
fig, ax = plt.subplots(1, 3, figsize=(20, 6))


ax[0].scatter(df4['x'], df4['y'], s=2)
ax[0].set_xlabel('X')
ax[0].set_ylabel('Y')
ax[0].grid()

ax[1].scatter(df4['y'], df4['z'], s=2)
ax[1].set_xlabel('Y')
ax[1].set_ylabel('Z')
ax[1].grid()

ax[2].scatter(df4['x'], df4['z'], s=2)
ax[2].set_xlabel('X')
ax[2].set_ylabel('Z')
ax[2].grid()

plt.show()

In [ ]:
def plot_spectrogram(df, top=None):
    winlen = 4096
    FS = 4096
    fig, ax = plt.subplots(4, 1, figsize=(20, 15))

    pxx, freq, t, cax = ax[0].specgram(
        df['x'], Fs=FS, 
        mode='magnitude',
        window=np.hanning(winlen), NFFT=winlen, noverlap=winlen//2
    )
    ax[0].set_xlabel('Time [s]')
    ax[0].set_ylabel('Frequency [Hz]')

    pxx, freq, t, cax = ax[1].specgram(
        df['y'], Fs=FS, 
        mode='magnitude',
        window=np.hanning(winlen), NFFT=winlen, noverlap=winlen//2
    )
    ax[1].set_xlabel('Time [s]')
    ax[1].set_ylabel('Frequency [Hz]')

    pxx, freq, t, cax = ax[2].specgram(
        df['z'], Fs=FS, 
        mode='magnitude',
        window=np.hanning(winlen), NFFT=winlen, noverlap=winlen//2
    )
    ax[2].set_xlabel('Time [s]')
    ax[2].set_ylabel('Frequency [Hz]')
    
    pxx, freq, t, cax = ax[3].specgram(
        df['mag'], Fs=FS, 
        mode='magnitude',
        window=np.hanning(winlen), NFFT=winlen, noverlap=winlen//2
    )
    ax[3].set_xlabel('Time [s]')
    ax[3].set_ylabel('Frequency [Hz]')

    plt.colorbar(cax, ax=ax)
    if top is not None:
        for i in range(3):
            ax[i].set_ylim(0, top)
    plt.show()

In [ ]:
plot_spectrogram(d0)

In [ ]:
plot_spectrogram(d4)

In [ ]:
rpm = d0['rpm'][int(t[100] * FS)]
print(f't = {t[100]} s = {rpm} RPM')

plt.plot(freq, pxx.T[100])
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude')
plt.yscale('log')
plt.grid()
plt.show()

In [ ]:
from scipy.fft import fft, ifft

WINDOW = 4096
OVERLAP = 0.5
STEP = WINDOW * OVERLAP
     
v = d0['mag'].to_numpy()
# (d0['f'].iloc[i:i+int(STEP)].mean(), 

# Frekvečné spektrum v okná
freqs = [int(i * (FS/WINDOW)) for i in range(WINDOW//2)] 
matrix = [
    np.abs(
        fft(
            d0['mag'].iloc[i:i+WINDOW].to_numpy() * 
            np.hamming(WINDOW)
           ) 
    )[:WINDOW // 2]
    for i in range(0, len(v) - WINDOW, int(STEP))
]
tf_spectrum = pd.DataFrame(data=matrix, columns=freqs)
tf_spectrum.index = (tf_spectrum.index * STEP) / FS
tf_spectrum

In [ ]:
slicetf = tf_spectrum.iloc[2][1:]
slicetf.plot(
    figsize=(20, 6), 
    xlabel='Frequency [Hz]', 
    ylabel='Amplitude', 
    logy=True,
    xlim=(1, 50),
    ylim=(0)
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 5))
src = tf_spectrum.iloc[2][2:]

ax = [ax]
ax[0].plot(src, label='8')
# ax[0].plot(np.convolve(src, np.ones(8) / 8, mode='full'), label='8')
# ax[0].plot(np.convolve(src, np.ones(16) / 16, mode='full'), label='16')
# ax[0].plot(np.convolve(src, np.ones(32) / 32, mode='full'), label='32')

plt.legend()

In [ ]:
# Priemery viacerých spektier na odhad PSD 
fig, ax = plt.subplots(3, 1, figsize=(20, 15))

ax[0].set_yscale('log')
ax[0].plot(
    tf_spectrum.iloc[:4].to_numpy().mean(axis=0)[2:]
)

ax[1].set_yscale('log')
ax[1].plot(
    tf_spectrum.iloc[:32].to_numpy().mean(axis=0)[2:]
)

ax[2].set_yscale('log')
ax[2].plot(
    tf_spectrum.iloc[:128].to_numpy().mean(axis=0)[2:]
)

In [ ]:
# Maximá 
fig, ax = plt.subplots(3, 1, figsize=(20, 15))

ax[0].set_yscale('log')
ax[0].plot(
    tf_spectrum.iloc[:4].to_numpy().max(axis=0)[2:]
)

ax[1].set_yscale('log')
ax[1].plot(
    tf_spectrum.iloc[:32].to_numpy().max(axis=0)[2:]
)

ax[2].set_yscale('log')
ax[2].plot(
    tf_spectrum.iloc[:128].to_numpy().max(axis=0)[2:]
)

In [ ]:
# Spočítaj Cepstrum
x = pd.DataFrame(data=[np.abs(fft(np.log(psd**2)))**2 for psd in tf_spectrum.to_numpy()])
plt.plot(x[1])

In [ ]:
# Spočítaj spektrálne koeficienty